In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plot
%matplotlib inline

In [17]:
features_df = pd.read_csv('./data/raw/X_train.csv')
target_df = pd.read_csv('./data/raw/y_train.csv')

In [18]:
print(features_df.shape)
features_df.head()

(59400, 40)


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [20]:
target_df.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [22]:
y = target_df['status_group'].replace({'functional': 0, 'non functional': 1, 'functional needs repair': 2}).astype(int)

In [23]:
target_df['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [25]:
y_train

24947    1
22630    0
13789    0
15697    0
22613    1
        ..
54343    0
38158    0
860      1
15795    0
56422    1
Name: status_group, Length: 44550, dtype: int64

In [26]:
import xgboost as xgb

In [27]:
model = xgb.XGBClassifier()

In [28]:
numerical_cols = list(X.select_dtypes(include=np.number).columns)
categorical_cols = list(X.select_dtypes(exclude=np.number).columns)

In [29]:
numerical_cols

['id',
 'amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'construction_year']

In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('scale', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [39]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
test_preds = my_pipeline.predict(X_test)
train_preds = my_pipeline.predict(X_train)

In [40]:
from sklearn.metrics import accuracy_score
print(f'test accuracy: {accuracy_score(y_test, test_preds)}')
print(f'train accuracy: {accuracy_score(y_train, train_preds)}')

test accuracy: 0.7903703703703704
train accuracy: 0.8205387205387206


In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_test, test_preds))

              precision    recall  f1-score   support

           0       0.77      0.92      0.84      8098
           1       0.85      0.70      0.77      5678
           2       0.67      0.26      0.37      1074

    accuracy                           0.79     14850
   macro avg       0.76      0.63      0.66     14850
weighted avg       0.79      0.79      0.78     14850



In [42]:
print(classification_report(y_train, train_preds))

              precision    recall  f1-score   support

           0       0.79      0.95      0.86     24161
           1       0.89      0.74      0.81     17146
           2       0.80      0.30      0.44      3243

    accuracy                           0.82     44550
   macro avg       0.83      0.66      0.70     44550
weighted avg       0.83      0.82      0.81     44550



In [ ]:
# 